# Fully Connected Network

In [1]:
import numpy as np
import pickle
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, GELU, Dropout

In [54]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [4]:
from sklearn.model_selection import StratifiedKFold

In [5]:
from torchinfo import summary

## Load data

In [6]:
DATA_FOLDER = '../data'
PICKLE_FOLDER = '../pickles'

In [7]:
with open(f'{PICKLE_FOLDER}/timeseries.pickle', 'rb') as f:
    ts = pickle.load(f)

In [8]:
total_samples, total_brain_regions, ts_length = ts.shape

In [9]:
df_metadata = pd.read_csv(f'{DATA_FOLDER}/patients-cleaned.csv', index_col=0)

In [10]:
df_metadata.head(2)

,age,sex,target
0,24.750,1,0
1,27.667,1,0


### Select connectivity dataset

In [11]:
THRESHOLD = 0.1                                          # 0.01, 0.05, 0.1, 0.15
N = 20 #                                                 # 3, 5, 7, 10, 15, 20, 40
CORR_TYPE = 'pearson'                                    # 'pearson', 'spearman', 'partial-pearson'
THRESHOLD_METHOD = 'abs-group-avg-diff'                  # 'abs-sample-diff', 'abs-group-avg-diff'
THRESHOLD_TYPE = 'min'                                 # 'min', 'max' or for kNN 'small', 'large'
KNN = False                                               # Whether all or only top N neigbors are taken

In [12]:
fc_folder = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}{"-knn" if KNN else ""}-{THRESHOLD_METHOD}'

In [13]:
fc_file_binary = f'{fc_folder}/{THRESHOLD_TYPE}-{f"knn-{N}" if KNN else f"th-{THRESHOLD}"}-binary.pickle'
fc_file_real = f'{fc_folder}/{THRESHOLD_TYPE}-{f"knn-{N}" if KNN else f"th-{THRESHOLD}"}-real.pickle'

In [14]:
with open(fc_file_binary, 'rb') as f:
    edge_index_matrix = pickle.load(f)

In [15]:
with open(fc_file_real, 'rb') as f:
    fc_matrix = pickle.load(f)

In [16]:
edge_index_matrix.shape

(190, 90, 90)

In [17]:
fc_matrix.shape

(190, 90, 90)

## Split data

In [18]:
with open(f'{PICKLE_FOLDER}/test-indices.pickle', 'rb') as f:
    test_indices = pickle.load(f)

In [19]:
train_indices = list(set(range(total_samples)) - set(test_indices))

In [20]:
train_targets = df_metadata.iloc[train_indices]["target"].reset_index(drop=True)

In [21]:
print(f'Train set size: {len(train_indices)}')
print(f'Test set size: {len(test_indices)}')

Train set size: 140
Test set size: 50


## Extend dataset

In [22]:
N_SURROGATES = 5                    # 5
SURROGATE_METHOD = 'iaaft'          # 'iaaft', 'aaft'

In [23]:
with open(f'{PICKLE_FOLDER}/timeseries-{SURROGATE_METHOD}-{N_SURROGATES}.pickle', 'rb') as f:
    ts_surrogates = pickle.load(f)

In [24]:
print(f'Extra data: {ts_surrogates.shape}')
print(f'{ts_surrogates.shape[0] / total_samples} surrogates per sample')

Extra data: (950, 90, 400)
5.0 surrogates per sample


Extend only training data. Test set will consist of true data only.

In [25]:
ts_train_surrogates = np.concatenate([ts_surrogates[N_SURROGATES*i:N_SURROGATES*i+N_SURROGATES] for i in train_indices], axis=0)

In [26]:
ts_train_surrogates.shape

(700, 90, 400)

## Prepare data

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### `Data` object fields

- `data.x`: Node feature matrix with shape `[num_nodes, num_node_features]`

- `data.edge_index`: Graph connectivity in COO format with shape `[2, num_edges]` and type `torch.long`

- `data.edge_attr`: Edge feature matrix with shape `[num_edges, num_edge_features]`

- `data.y`: Target to train against (may have arbitrary shape), e.g., node-level targets of shape `[num_nodes, *]` or graph-level targets of shape `[1, *]`

- `data.pos`: Node position matrix with shape `[num_nodes, num_dimensions]`

### Select node features

- onehot
- timeseries
- correlations

In [28]:
ts_length_start = 50
ts_length_end = ts_length

# A part of signal serves as node features.
def timeseries_in_nodes(i):
    return torch.from_numpy(ts[i][:,ts_length_start:ts_length_end]).to(torch.float32)

In [29]:
# Each nodes contains its row from FC matrix.
def correlations_in_nodes(i):
    return torch.from_numpy(fc_matrix[i].reshape(-1)).to(torch.float32)

In [30]:
# Each brain region is onehot encoded. See GIN for phenotype paper.
def onehot_in_nodes(i):
    return torch.diag(torch.ones(total_brain_regions))

In [31]:
features_in_nodes = correlations_in_nodes
num_features_in_nodes = total_brain_regions**2         # total_brain_regions / ts_length_end-ts_end_start

### Create dataset

In [32]:
dataset = [(
    features_in_nodes(i).to(device), 
    torch.tensor(target, dtype=torch.int64).to(device)
) for target, i in zip(train_targets, train_indices)]

In [33]:
print(f'True train data: {len(dataset)}')

True train data: 140


In [35]:
# Extension.
dataset_ext = [(
    features_in_nodes(i).to(device),  
    torch.tensor([target], dtype=torch.int64).to(device)
) for target, i, ts_surr in zip(
    np.repeat(train_targets, N_SURROGATES),
    np.repeat(train_indices, N_SURROGATES),
    ts_train_surrogates
)]

In [36]:
print(f'Surrogate train data: {len(dataset_ext)}')

Surrogate train data: 700


### Join surrogates with true data

In [37]:
# dataset.extend(dataset_ext)

# train_targets_ext = np.concatenate([train_targets, np.repeat(train_targets, N_SURROGATES)])
# train_targets_ext.shape

In [38]:
print(f'Extended train data: {len(dataset)}')

Extended train data: 140


In [39]:
print('Data object')
print(f'Features: {dataset[0][0].shape}')
print(f'Target: {dataset[0][1].shape}')

Data object
Features: torch.Size([8100])
Target: torch.Size([])


## Define model

In [164]:
class FC(torch.nn.Module):
    
    def __init__(self, hidden_channels, p_dropout=0.5):
        super(FC, self).__init__()
        
        self.fc1 = Linear(num_features_in_nodes, hidden_channels)
        self.activation1 = GELU()
        self.dropout1 = Dropout()

        self.fc2 = Linear(hidden_channels, hidden_channels)
        self.activation2 = GELU()
        self.dropout2 = Dropout()

        self.fc3 = Linear(hidden_channels, hidden_channels)
        self.activation3 = GELU()
        self.dropout3 = Dropout()

        # self.fc4 = Linear(num_features_in_nodes+hidden_channels*3, 2)
        self.fc4 = Linear(hidden_channels, 2)

    def forward(self, x):

        # 1. Obtain node embeddings
        # 2. Remember layer outputs.
        out0 = x
        x = self.dropout1(self.activation1(self.fc1(x)))
        out1 = x
        x = self.dropout2(self.activation2(self.fc2(x)))
        out2 = x
        x = self.dropout3(self.activation3(self.fc3(x)))
        out3 = x

        # 3. Concatenate itermediate outputs.
        # x = torch.cat((out0, out1, out2, out3), 1)

        # 4. Fully connected for final classification.
        x = self.fc4(x)     # CELoss already incorporates `log_softmax`.
        
        return x

In [165]:
summary(FC(8))

Layer (type:depth-idx)                   Param #
FC                                       --
├─Linear: 1-1                            64,808
├─GELU: 1-2                              --
├─Dropout: 1-3                           --
├─Linear: 1-4                            72
├─GELU: 1-5                              --
├─Dropout: 1-6                           --
├─Linear: 1-7                            72
├─GELU: 1-8                              --
├─Dropout: 1-9                           --
├─Linear: 1-10                           18
Total params: 64,970
Trainable params: 64,970
Non-trainable params: 0

## Evaluation

In [166]:
def evaluation_metrics(predicted, labels):
    pred_positives = predicted == 1
    label_positives = labels == 1

    tp = (pred_positives & label_positives).sum().item()
    tn = (~pred_positives & ~label_positives).sum().item()
    fp = (pred_positives & ~label_positives).sum().item()
    fn = (~pred_positives & label_positives).sum().item()

    return tp, tn, fp, fn

## Train model

In [174]:
NUM_FOLDS = 7

In [175]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, random_state=42, shuffle=True)

In [176]:
# Settings.
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.5
OPTIMIZER = 'adam'
LOSS = 'ce'
BATCH_SIZE = 8

VALIDATE_FREQ = 1

HIDDEN_CHANNELS = 8
DROPOUT = 0.5

STEP_SIZE = 50
GAMMA = 0.5

WEIGHT_DECAY = 0.0001

settings_str = f'bs={BATCH_SIZE},e={EPOCHS},lr={LR},mom={MOMENTUM},opt={OPTIMIZER},loss={LOSS},step={STEP_SIZE},gamma={GAMMA},wd={WEIGHT_DECAY},dropout={DROPOUT}'

In [177]:
lr_lambda = lambda e: 0.01 if e < 200 else (0.001 if e < 300 else 0.0001)

In [178]:
# Experiment folder.
EXP_FOLDER = 'runs/fc'

In [179]:
# Experiment.
EXP_ID = 13

In [180]:
for kfold, (train_index, val_index) in enumerate(skf.split(np.zeros(len(train_targets)), train_targets)):

    # Init TB writer.
    experiment_str = f'id={EXP_ID:03d},fold={kfold},{settings_str}'
    writer = SummaryWriter(f"../{EXP_FOLDER}/{experiment_str}")

    # Init model.
    net = FC(hidden_channels=HIDDEN_CHANNELS, p_dropout=DROPOUT).to(device)
    optimizr = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    # optimizr = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

    criterion = torch.nn.CrossEntropyLoss()
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizr, step_size=STEP_SIZE, gamma=GAMMA)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizr, lr_lambda)

    # Save architecture.
    with open(f"../{EXP_FOLDER}/{experiment_str}/architecture", 'w', encoding="utf-8") as f:
        f.write(fc_folder + '\n')
        f.write(fc_file_binary + '\n')
        f.write(fc_file_real + '\n')
        f.write(features_in_nodes.__str__() + '\n')
        f.write('\n'.join(experiment_str.split(',')) + '\n\n')
        f.write(net.__str__() + '\n\n')
        f.write(str(summary(net)))

    # Prepare data.
    X_train = [dataset[i] for i in train_index]
    X_val = [dataset[i] for i in val_index]
    
    trainloader = DataLoader(X_train, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(X_val, batch_size=BATCH_SIZE, shuffle=False)

    # Train.
    for epoch in range(EPOCHS):
        running_loss = 0.
        
        net.train()
        for x, y in trainloader:
            
            optimizr.zero_grad()
            outputs = net(x)   
            loss = criterion(outputs, y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizr.step()
            running_loss += loss.item()

        #scheduler.step()    # Update LR.
        epoch_loss = running_loss / len(trainloader)
        writer.add_scalar('training loss', epoch_loss, epoch)

        running_loss = 0.

        # Evaluate epoch.
        tp, tn, fp, fn = 0, 0, 0, 0
        total = 0

        net.eval()
        for x, y in valloader:

            optimizr.zero_grad()
            outputs = net(x) 
            loss = criterion(outputs, y)  # Compute the loss.
            running_loss += loss.item()

            if (epoch+1) % VALIDATE_FREQ == 0:
                predicted = outputs.argmax(dim=1)
                labels = y.view(-1)

                # Update.
                tp_, tn_, fp_, fn_ = evaluation_metrics(predicted, labels)
                tp += tp_; tn += tn_; fp += fp_; fn += fn_
                total += y.size(0)

        epoch_loss = running_loss / len(valloader)
        writer.add_scalar('validation loss', epoch_loss, epoch)
        
        if (epoch+1) % VALIDATE_FREQ == 0:
            writer.add_scalar('validation accuracy', (tp + tn) / total, epoch)
            writer.add_scalar('validation precision', tp / (tp + fp) if (tp + fp) > 0 else 0, epoch)
            writer.add_scalar('validation recall', tp / (tp + fn), epoch)
    
    # Single fold during exploration.
    #break

print('Finished training')

Finished training
